# Import

In [1]:
import os

os.system("pip install --upgrade -q pip")
os.system("pip install --upgrade -q sagemaker")
os.system("pip install --upgrade -q boto3")
os.system("pip install -q mlflow")
os.system("pip install -q sagemaker-mlflow")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.3.1 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.40.16 which is incompatible.
autogluon-multimodal 1.3.1 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-multimodal 1.3.1 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.53.1 which is incompatible.
autogluon-timeseries 1.3.1 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.53.1 which is incompatible.
sagemaker-studio-analytics-extension 0.2.0 requires sparkmagic==0.22.0, but you have sparkmagic 0.21.0 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.3 which is incompatible.


0

In [45]:
import boto3
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role

from sagemaker.sklearn.estimator import SKLearn
from sagemaker.tuner import IntegerParameter, HyperparameterTuner
from sagemaker.model import Model

# SageMaker Setting

In [3]:
boto_session = boto3.Session()
sagemaker_session = Session(default_bucket='mlflow-artifacts-dir', boto_session=boto_session)
role = get_execution_role()

# Define Estimator

In [38]:
param_dict = {
    'n_estimators' : 200,
    'max_depth' : 7,
    'max_features' : 10,
    'experiment_name' : 'Iris-Project',
    'run_name' : 'rf-clf-exp-2'
}

estimator = SKLearn(
    entry_point='train.py',
    source_dir='./script',
    role=role,
    framework_version='1.2-1',
    py_version='py3',
    instance_type='ml.m5.xlarge',
    instance_count=1,
    model_dir='/opt/ml/model',
    hyperparameters=param_dict
)

# Define HyperparameterTuner

In [39]:
param_range = {
    'n_estimators' : IntegerParameter(100, 300),
    'max_depth' : IntegerParameter(3, 9),
    'max_features' : IntegerParameter(3, 10),
}

tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name='test accuracy',
    objective_type='Maximize',
    max_jobs=30,
    max_parallel_jobs=3,
    hyperparameter_ranges=param_range,
    metric_definitions=[
        {
            'Name' : 'test accuracy',
            'Regex' : 'Test Accuracy : ([0-9\\.]+)'
        }
    ]
)

# Tuning Estimator Using HyperparameterTuner

In [40]:
channels = {
    'trainig' : 's3://mlflow-artifacts-dir/iris',
    'testing' : 's3://mlflow-artifacts-dir/iris'
}

In [41]:
tuner.fit(inputs=channels)

INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-scikit-lea-250825-0507


........................................................................................................................................................................................................!


# Regist Best Model

In [46]:
best_estimator = tuner.best_estimator()

model = Model(
    model_data=best_estimator.model_data,
    image_uri=best_estimator.image_uri,
    role=role,
    sagemaker_session=sagemaker_session
)

model_package = model.register(
    model_package_group_name="Iris-RF-CLF",
    approval_status="PendingManualApproval"
)


2025-08-25 05:10:59 Starting - Preparing the instances for training
2025-08-25 05:10:59 Downloading - Downloading the training image
2025-08-25 05:10:59 Training - Training image download completed. Training in progress.
2025-08-25 05:10:59 Uploading - Uploading generated training model
2025-08-25 05:10:59 Completed - Resource reused by training job: sagemaker-scikit-lea-250825-0507-005-ee120442
